In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from text2vec import SentenceModel
# model = SentenceModel('GanymedeNil/text2vec-large-chinese')
model = SentenceModel('./outputs/Product_Sim/checkpoint-540-epoch-30')

2023-07-31 09:25:25.373 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cuda


In [9]:
sentences = ["鞭炮"] + "繁殖能力的植物,宗教祭祀,中药材".split(",")
sentence_embeddings = model.encode(sentences)
cosine_similarity(sentence_embeddings)

array([[ 0.9999999 ,  0.22484937,  0.34070042, -0.10219967],
       [ 0.22484937,  0.99999976,  0.40718102,  0.05660292],
       [ 0.34070042,  0.40718102,  0.99999964,  0.10734674],
       [-0.10219967,  0.05660292,  0.10734674,  0.9999997 ]],
      dtype=float32)

In [10]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("shalomma/llama-7b-embeddings", trust_remote_code=True)
model = AutoModel.from_pretrained("shalomma/llama-7b-embeddings", trust_remote_code=True).half().to("cuda:0").eval()



ValueError: Tokenizer class LLaMATokenizer does not exist or is not currently imported.

Bad pipe message: %s [b'\xfe\x87\xbaI\xb7\xc9\x10\xc1\xe3\xeb\xe8\x91\xc4\x0e4"\xff\x9e "P\xde\xda@\xc2\xe3\x16\xfb\xc7\xabR\xae[\x91\xdb\xf9\xd3v\x89\x18\xe7a\x89(\x91*m\x17\xa0\x1f\x11\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e']
Bad pipe message: %s [b'\xd1\xa2\x86\x19~?']
Bad pipe message: %s [b"\x9e8\xf0V\xe3\x14\x10\x99\xc9U\xc7\x13c5\x19\xd3\xcc\x03\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from sklearn.metrics.pairwise import cosine_similarity



def get_glm_embedding(text, device="cuda:0", logits=False):
    inputs = tokenizer([text], return_tensors="pt").to(device)
    resp = model.transformer(**inputs, output_hidden_states=True)
    y = resp.last_hidden_state
    if logits:
        y = model.lm_head(y).permute(1, 0, 2).contiguous()
    y = y.squeeze()
    y_mean = torch.mean(y, dim=0, keepdim=True)
    return y_mean.cpu().detach().numpy()[0]


sentences = ["鞭炮"] + "繁殖能力的植物,宗教祭祀,中药材".split(",")
sentence_embeddings = [get_glm_embedding(text) for text in sentences]
cosine_similarity(sentence_embeddings)

array([[1.        , 0.77742075, 0.77483627, 0.8067396 ],
       [0.77742075, 1.        , 0.79073598, 0.80767237],
       [0.77483627, 0.79073598, 1.        , 0.79779014],
       [0.8067396 , 0.80767237, 0.79779014, 1.        ]])

In [13]:
import pandas as pd
df = pd.read_csv("Product_Sim.csv", encoding='utf-8', sep='\t', names=["text1", "text2", "score"])
df.head()

,text1,text2,score
0,鞭炮,烟花,1
1,鞭炮,爆竹,1
2,鞭炮,汇丰炮,1
3,鞭炮,电子鞭炮,1
4,鞭炮,鞭炮棒,1


In [15]:
from sklearn.model_selection import train_test_split

ratio = 0.4
n_samples = df.shape[0]

df_train, df_test = train_test_split(df, test_size=ratio, random_state=42)
df_test, df_valid = train_test_split(df_test, test_size=0.5, random_state=42)
df_train.to_csv("Product_Sim_Train.csv", encoding='utf-8', sep='\t', header=None, index=False)
df_test.to_csv("Product_Sim_Test.csv", encoding='utf-8', sep='\t', header=None, index=False)
df_valid.to_csv("Product_Sim_Valid.csv", encoding='utf-8', sep='\t', header=None, index=False)

In [6]:
df_valid.head()

,text1,text2,score
3038,羊肝,有解毒作用,1
2132,鲜花,仪仗,0
544,梨,雪茄,0
932,葡萄酒,酒精饮料,1
3346,香蕉,橙子,1
